# 使用`torch.autograd`自动微分
当训练一个神经网络时，最常用的方法就是反向传播(`back propagetion`)算法。在这个算法中，模型中的参数都基于损失函数的梯度来进行调整。为了计算这些梯度，PyTorch提供了内置引擎`torch.autograd`。它能够用来自动计算任意计算图(`computational graph`)的微分。可以使用以下步骤来建立一个单层的(输入`x`,权重`w`,偏移`b`)神经网络模型。


In [19]:
import torch
x = torch.ones(5)
y = torch.ones(3)
w = torch.randn(5,3,requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

# Tensors, Functions and Computational graph
## tensor.requires_grad 属性
上面的code定义了下面的**计算图**
![](https://pytorch.org/tutorials/_images/comp-graph.png)
在上面这个网络中，`w` 和 `b`是参数,是需要进行优化的。因此，我们需要能够关于这些参数来计算梯度,(在数学里面，也就是以这些参数作为求导数或者微分的因子)。为了实现微分操作，需要给上述的`tensor`设置属性`requires_grad`值。
> 可以在创建tensor是使用`requires_grad`属性值或者创建完之后使用`x.requires_grad_(True)`方法。

## Functions类
用来应用到tensor的，创建计算图的函数其实是`Function`类的一个子类，这个对象掌握了计算前向传播过程在的参数和反向传播的导数。关于反向传播的操作都被纪律在`tensor`的`grad_fn`属性中。

In [20]:
print('z的梯度函数',z.grad_fn)
print('loss的梯度函数',loss.grad_fn)

z的梯度函数 <AddBackward0 object at 0x7fd841f2ab90>
loss的梯度函数 <BinaryCrossEntropyWithLogitsBackward object at 0x7fd841f2ac10>


为了优化模型中的这些参数，我们需要基于损失函数来计算这些参数(`w`和`b`)的导数。也就是说，我们需要在一些固定的`x`和`y`下计算 $\frac{\partial loss}{\partial w}$ and
$\frac{\partial loss}{\partial b}$这两个偏导数。这就是所谓的**反向传播**，然后可以从`w.grad` 和 `b.grad`两个函数中来检索这些值。

In [21]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-0.1650, -0.2951, -0.3016],
        [-0.1650, -0.2951, -0.3016],
        [-0.1650, -0.2951, -0.3016],
        [-0.1650, -0.2951, -0.3016],
        [-0.1650, -0.2951, -0.3016]])
tensor([-0.1650, -0.2951, -0.3016])


> 1. 我们只能在计算图的叶子节点处获得`grad`属性值，而且还需要其设置`requires_grad=True`,对于其他节点，都是无法获得此属性的。
> 2. 我们仅仅可以在一个计算图上执行一次反向传播（即只能调用一次`backward()`），为了计算需要，如果我们需要做多次反向传播(即调用多次`backward()`函数)，那么需要给反向传播函数设置`retain_graph=True`属性值。

# 禁用梯度跟踪
默认情况下，所有的设置了`requires_grad=True`的tensor,都会跟踪存储他们的计算历史，还会支持梯度计算。然而有时候我们并不需要跟踪计算的历史，举个例子，当我们已经将一个模型训练好，仅仅需要应用这个模型到一个输入数据中。我仅仅需要计算前向传播，可以通过使用`torch.no_grad()`代码块来避免计算和跟踪计算历史。也可以在tensor上调用detach()方法

In [22]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


In [23]:
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


## 其他禁用梯度跟踪的理由
1. 对一些预训练好的网络进行微调时，需要将一些参数设置为*冻结参数*
2. 当仅仅需要前向传播计算时，可以禁用反向传播，**用以加速计算**

# 计算图流程
从概念上说，自动微分(autograd)保留了在有向无环图计算过程中的，所有的tensor数据和在这些数据上的操作，包括结果和Function对象。叶子节点是输入节点，根节点是输出节点，通过跟踪从根节点到叶子节点的数据，可以使用链式法则自动计算梯度。
## 前向传播
在前向传播过程中，（叶子节点到根节点），自动微分同时做两件事。
+ 执行要求的操作来得到目标tensor
+ 管理维护在DAG中的这些操作的**梯度函数**

当反向传播函数(`.backward()`)在计算图的根节点(`root`)被调用，自动微分执行以下操作:
+ 从每个 `.grad_fn`函数计算梯度
+ 将它们累积到各自tensor的`.grad`属性中
+ 使用链式法则，一路传播到叶子节点进行计算。

> ** 在PyTorch中，计算图(`DAGs`)是动态的。**,计算图是会在根节点重建的，在每次`.backward()`方法被调用之后，`autograd`都会重建一个新的计算图。这也就表示我们可以在图的重建过程给模型中添加任何逻辑操作描述。如果需要可以在任何一次迭代中改变维度，大小。

# Tensor梯度和Jacobian矩阵
在许多情况下，我们有许多的损失函数，我们需要去基于这些参数计算梯度。然后，有时候输出函数是任意的tensor，此时，PyTorch允许你区计算所谓的雅各比矩阵(**Jacobian product**),而不是实际的梯度。
对于一个向量函数 $\vec{y}=f(\vec{x})$, 当
$\vec{x}=\langle x_1,\dots,x_n\rangle$ 且
$\vec{y}=\langle y_1,\dots,y_m\rangle$, 
$\vec{y}$ 关于 $\vec{x}$ 的梯度就是雅各比矩阵(**Jacobian
matrix**):
\begin{align}J=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}
可以不计算雅各比矩阵本身，PyTorch允许对于一个给定的向量$v=(v_1 \dots v_m)$计算**Jacobian Product** $v^T\cdot J$。可以通过给`.backward()`函数传递`v`参数来实现，这个$v$的大小应该和我们需要计算其矩阵的原tensor一样的大小。

In [24]:
inp = torch.eye(5, requires_grad=True)  # 可以计算梯度
out = (inp+1).pow(2) # 计算二次幂
out.backward(torch.ones_like(inp), retain_graph=True) # 一次反向传播，传递的d第一个参数就是v，与inp是一样的大小，并设置可重复计算。
print("First call\n", inp.grad)  # 返回梯度值
out.backward(torch.ones_like(inp), retain_graph=True) # 二次反向传播，没有初始化，梯度会累积
print("\nSecond call\n", inp.grad)
inp.grad.zero_()  # 就地0化
out.backward(torch.ones_like(inp), retain_graph=True)  # 三次反向传播
print("\nCall after zeroing gradients\n", inp.grad) # 返回计算的梯度值

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


由上可以看出，每一次反向传播的过程中，值都是不同的。这是因为当进行`backward`计算时，PyTorch 积累了梯度。举个例子，计算得到的梯度的值被加到了对应的叶子节点的`grad`属性中，如果想要计算出合适的梯度，需要0化这些梯度。我们也可以使用*optimizer*来实现这个操作。
> 在前面的代码里面，我们都是直接调用`backwark()`函数而没有参数，这本质上其实就是调用了`backward(torch.tensor(1.0))`方法，当计算纯量值时，就比如说在训练过程中的损失值，指定参数的方法时十分游泳的。

Over